In [1]:
import os
import pandas as pd
import sys
import numpy as np
import sys
sys.path.insert(0, '../..')
import itertools

from JKBio.utils import helper as h
from JKBio.epigenetics import chipseq as chip

import dalmatian as dm
import pyBigWig

from bokeh.plotting import *
from IPython.display import IFrame
import igv
from biomart import BiomartServer
import io

from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture

output_notebook()
%load_ext autoreload
%autoreload 2

Loading BokehJS ...

In [25]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
url="https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U"
gsheet = sheets.get(url).sheets[2].to_frame()

In [7]:
gsheet

,id,cell line,replicate,protein,quality,paired_end,matching input name,processed,name,previous name,...,unique mapped reads(droso),scaling factor,Unnamed: 25,Total QC,folderNarrow,folderCompensated,folderQC,folderBroad,folder Bigwig,folder diffPeaks
0,mp100,U937,1,INPUT,NaN,n,NaN,Y,mp100-U937-INPUT-r1,NaN,...,NaN,NaN,NaN,https://storage.cloud.google.com/amlproject/Ch...,https://console.cloud.google.com/storage/brows...,NaN,https://console.cloud.google.com/storage/brows...,https://console.cloud.google.com/storage/brows...,https://console.cloud.google.com/storage/brows...,NaN
1,mp101,NOMO1,1,INPUT,NaN,n,NaN,Y,mp101-NOMO1-INPUT-r1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,mp102,UT7,1,INPUT,NaN,n,NaN,Y,mp102-UT7-INPUT-r1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mp106,MV411,1,MYB,x,n,mp99,Y,mp106-MV411-MYB-r1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mp109,M6,1,CEBPA,x,n,mp98,Y,mp109-M6-CEBPA-r1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,mp916,MV411,1,E2F3,NaN,y,mp845,NaN,mp916-MV411-E2F3-r1,20201023_E2F3_MP8382_S41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
374,mp917,MV411,1,SREBP1,NaN,y,mp845,NaN,mp917-MV411-SREBP1-r1,20201023_SREBP1_MP8382_S42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
375,mp918,MV411_GFI_SC,1,FLAG_GFI,NaN,y,mp845,NaN,mp918-MV411_GFI_SC-FLAG_GFI-r1,20200903_1_MP8215_S101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
376,mp919,MV411_MEF2D_NT_SC_63,3,FLAF_MEF2D,NaN,y,mp845,NaN,mp919-MV411_MEF2D_NT_SC_63-FLAF_MEF2D-r3,20200903_2_MP8215_S102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## v10

In [1]:
project="chipseq_MEF2C_2h_degraded"
loc1= "201222_MP8548_fastq"
loc2= "201204_MP8489_fastq"

In [15]:
fastq2 = ! gsutil ls gs://transfer-amlproject/$loc2/
fastq2

['gs://transfer-amlproject/201204_MP8489_fastq/20201204_DMSO_FLAG_MEF2D_degron_2h_MP8489_S47_R1_001.fastq.gz',
 'gs://transfer-amlproject/201204_MP8489_fastq/20201204_DMSO_FLAG_MEF2D_degron_2h_MP8489_S47_R2_001.fastq.gz',
 'gs://transfer-amlproject/201204_MP8489_fastq/20201204_DMSO_MED1_MEF2D_degron_2h_MP8489_S49_R1_001.fastq.gz',
 'gs://transfer-amlproject/201204_MP8489_fastq/20201204_DMSO_MED1_MEF2D_degron_2h_MP8489_S49_R2_001.fastq.gz',
 'gs://transfer-amlproject/201204_MP8489_fastq/20201204_DMSO_MEF2C_MEF2D_degron_2h_MP8489_S51_R1_001.fastq.gz',
 'gs://transfer-amlproject/201204_MP8489_fastq/20201204_DMSO_MEF2C_MEF2D_degron_2h_MP8489_S51_R2_001.fastq.gz',
 'gs://transfer-amlproject/201204_MP8489_fastq/20201204_DMSO_MYC_MEF2D_degron_2h_MP8489_S48_R1_001.fastq.gz',
 'gs://transfer-amlproject/201204_MP8489_fastq/20201204_DMSO_MYC_MEF2D_degron_2h_MP8489_S48_R2_001.fastq.gz',
 'gs://transfer-amlproject/201204_MP8489_fastq/20201204_DMSO_POL_II_MEF2D_degron_2h_MP8489_S50_R1_001.fastq.gz',

In [17]:
fastq1 = ! gsutil ls gs://transfer-amlproject/$loc1/
fastq1

['gs://transfer-amlproject/201222_MP8548_fastq/20201222_FLAG_DMSO_2h_MP8548_S1_R1_001.fastq.gz',
 'gs://transfer-amlproject/201222_MP8548_fastq/20201222_FLAG_DMSO_2h_MP8548_S1_R2_001.fastq.gz',
 'gs://transfer-amlproject/201222_MP8548_fastq/20201222_FLAG_VHL_2h_MP8548_S6_R1_001.fastq.gz',
 'gs://transfer-amlproject/201222_MP8548_fastq/20201222_FLAG_VHL_2h_MP8548_S6_R2_001.fastq.gz',
 'gs://transfer-amlproject/201222_MP8548_fastq/20201222_MED1_DMSO_2h_MP8548_S3_R1_001.fastq.gz',
 'gs://transfer-amlproject/201222_MP8548_fastq/20201222_MED1_DMSO_2h_MP8548_S3_R2_001.fastq.gz',
 'gs://transfer-amlproject/201222_MP8548_fastq/20201222_MED1_VHL_2h_MP8548_S8_R1_001.fastq.gz',
 'gs://transfer-amlproject/201222_MP8548_fastq/20201222_MED1_VHL_2h_MP8548_S8_R2_001.fastq.gz',
 'gs://transfer-amlproject/201222_MP8548_fastq/20201222_MEF2C_DMSO_2h_MP8548_S5_R1_001.fastq.gz',
 'gs://transfer-amlproject/201222_MP8548_fastq/20201222_MEF2C_DMSO_2h_MP8548_S5_R2_001.fastq.gz',
 'gs://transfer-amlproject/20122

In [5]:
mkdir ../data/$project/ && mkdir ../data/$project/qc/

In [7]:
!gsutil -m cp gs://transfer-amlproject/$loc1/multiqc_report.html ../data/$project/qc/
!gsutil -m cp -r gs://transfer-amlproject/$loc1/Reports/ ../data/$project/qc/
!gsutil -m cp -r gs://transfer-amlproject/$loc1/multiqc_data/ ../data/$project/qc/
    
!gsutil -m cp gs://transfer-amlproject/$loc2/multiqc_report.html ../data/$project/qc2/
!gsutil -m cp -r gs://transfer-amlproject/$loc2/Reports/ ../data/$project/qc2/
!gsutil -m cp -r gs://transfer-amlproject/$loc2/multiqc_data/ ../data/$project/qc2/

Copying gs://transfer-amlproject/201222_MP8548_fastq/multiqc_report.html...
/ [1/1 files][  1.3 MiB/  1.3 MiB] 100% Done                                    
Operation completed over 1 objects/1.3 MiB.                                      
Copying gs://transfer-amlproject/201222_MP8548_fastq/Reports/20201222_FLAG_DMSO_2h_MP8548_S1_R1_001_fastqc.html...
Copying gs://transfer-amlproject/201222_MP8548_fastq/Reports/20201222_FLAG_DMSO_2h_MP8548_S1_R1_001_fastqc.zip...
Copying gs://transfer-amlproject/201222_MP8548_fastq/Reports/20201222_FLAG_DMSO_2h_MP8548_S1_R2_001_fastqc.html...
Copying gs://transfer-amlproject/201222_MP8548_fastq/Reports/20201222_FLAG_DMSO_2h_MP8548_S1_R2_001_fastqc.zip...
Copying gs://transfer-amlproject/201222_MP8548_fastq/Reports/20201222_FLAG_VHL_2h_MP8548_S6_R1_001_fastqc.html...
Copying gs://transfer-amlproject/201222_MP8548_fastq/Reports/20201222_FLAG_VHL_2h_MP8548_S6_R1_001_fastqc.zip...
Copying gs://transfer-amlproject/201222_MP8548_fastq/Reports/20201222_FLAG_V

CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: Destination URL must name a directory, bucket, or bucket
subdirectory for the multiple source form of the cp command.
CommandException: Destination URL must name a di

In [8]:
! gsutil -m cp gs://transfer-amlproject/$loc1/*  gs://transfer-amlproject/$project/
! gsutil -m cp gs://transfer-amlproject/$loc2/*  gs://transfer-amlproject/$project/

Omitting prefix "gs://transfer-amlproject/201222_MP8548_fastq/Reports/". (Did you mean to do cp -r?)
Omitting prefix "gs://transfer-amlproject/201222_MP8548_fastq/multiqc_data/". (Did you mean to do cp -r?)
Copying gs://transfer-amlproject/201222_MP8548_fastq/20201222_FLAG_DMSO_2h_MP8548_S1_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/201222_MP8548_fastq/20201222_FLAG_DMSO_2h_MP8548_S1_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/201222_MP8548_fastq/20201222_FLAG_VHL_2h_MP8548_S6_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/201222_MP8548_fastq/20201222_MED1_DMSO_2h_MP8548_S3_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/201222_MP8548_fastq/20201222_FLAG_VHL_2h_MP8548_S6_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/201222_MP8548_fastq/20201222_MED1_DMSO_2h_

### analysis

In [11]:
rename = {
"20201222_FLAG_DMSO_2h_MP8548_S1": "mp931-MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D-r5",
"20201222_FLAG_VHL_2h_MP8548_S6": "mp932-MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D-r5",
"20201222_MED1_DMSO_2h_MP8548_S3": "mp933-MV411_MEF2D_NT_SC_63_DMSO-MED1-r3",
"20201222_MED1_VHL_2h_MP8548_S8": "mp934-MV411_MEF2D_NT_SC_63_VHL_2h-MED1-r3",
"20201222_MEF2C_DMSO_2h_MP8548_S5": "mp935-MV411_MEF2D_NT_SC_63_DMSO-MEF2C-r3",
"20201222_MEF2C_VHL_2h_MP8548_S10": "mp936-MV411_MEF2D_NT_SC_63_VHL_2h-MEF2C-r3",
"20201222_MYC_DMSO_2h_MP8548_S2": "mp937-MV411_MEF2D_NT_SC_63_DMSO-MYC-r3",
"20201222_MYC_VHL_2h_MP8548_S7": "mp938-MV411_MEF2D_NT_SC_63_VHL_2h-MYC-r3",
"20201222_POL_II_DMSO_2h_MP8548_S4": "mp939-MV411_MEF2D_NT_SC_63_DMSO-POLII_total-r3",
"20201222_POL_II_VHL_2h_MP8548_S9": "mp940-MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total-r3",
"20201204_DMSO_FLAG_MEF2D_degron_2h_MP8489_S47": "mp941-MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D-r6",
"20201204_DMSO_MED1_MEF2D_degron_2h_MP8489_S49": "mp942-MV411_MEF2D_NT_SC_63_DMSO-MED1-r4",
"20201204_DMSO_MEF2C_MEF2D_degron_2h_MP8489_S51": "mp943-MV411_MEF2D_NT_SC_63_DMSO-MEF2C-r4",
"20201204_DMSO_MYC_MEF2D_degron_2h_MP8489_S48": "mp944-MV411_MEF2D_NT_SC_63_DMSO-MYC-r4",
"20201204_DMSO_POL_II_MEF2D_degron_2h_MP8489_S50": "mp945-MV411_MEF2D_NT_SC_63_DMSO-POLII_total-r4",
"20201204_VHL_FLAG_MEF2D_degron_2h_MP8489_S52": "mp946-MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D-r6",
"20201204_VHL_MED1_MEF2D_degron_2h_1_MP8489_S54": "mp947-MV411_MEF2D_NT_SC_63_VHL_2h-MED1-r4",
"20201204_VHL_MED1_MEF2D_degron_2h_2_MP8489_S56": "mp948-MV411_MEF2D_NT_SC_63_VHL_2h-MEF2C-r4",
"20201204_VHL_MYC_MEF2D_degron_2h_MP8489_S53": "mp949-MV411_MEF2D_NT_SC_63_VHL_2h-MYC-r4",
"20201204_VHL_POL_II_MEF2D_degron_2h_MP8489_S55": "mp950-MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total-r4",}

In [18]:
for val in fastq1+fastq2:
    rep = val
    for k,v in rename.items():
        rep = rep.replace(k,v)
    rep = rep.replace('transfer-amlproject/'+loc1+'/',"amlproject/Chip/"+project+'/fastqs/').replace('transfer-amlproject/' + loc2 + '/', "amlproject/Chip/"+project+'/fastqs/')
    !gsutil cp $val $rep

Copying gs://transfer-amlproject/201222_MP8548_fastq/20201222_FLAG_DMSO_2h_MP8548_S1_R1_001.fastq.gz [Content-Type=application/octet-stream]...
/ [1 files][  2.4 GiB/  2.4 GiB]                                                
Operation completed over 1 objects/2.4 GiB.                                      
Copying gs://transfer-amlproject/201222_MP8548_fastq/20201222_FLAG_DMSO_2h_MP8548_S1_R2_001.fastq.gz [Content-Type=application/octet-stream]...
- [1 files][  2.4 GiB/  2.4 GiB]                                                
Operation completed over 1 objects/2.4 GiB.                                      
Copying gs://transfer-amlproject/201222_MP8548_fastq/20201222_FLAG_VHL_2h_MP8548_S6_R1_001.fastq.gz [Content-Type=application/octet-stream]...
/ [1 files][  2.0 GiB/  2.0 GiB]                                                
Operation completed over 1 objects/2.0 GiB.                                      
Copying gs://transfer-amlproject/201222_MP8548_fastq/20201222_FLAG_VHL_2h_MP8548

In [19]:
! gsutil rm gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/fastqs/multiqc_report.html

Removing gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/fastqs/multiqc_report.html...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              


In [20]:
fastq = ! gsutil ls gs://amlproject/Chip/$project/fastqs/
fastq

['gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/fastqs/mp931-MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D-r5_R1_001.fastq.gz',
 'gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/fastqs/mp931-MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D-r5_R2_001.fastq.gz',
 'gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/fastqs/mp932-MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D-r5_R1_001.fastq.gz',
 'gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/fastqs/mp932-MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D-r5_R2_001.fastq.gz',
 'gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/fastqs/mp933-MV411_MEF2D_NT_SC_63_DMSO-MED1-r3_R1_001.fastq.gz',
 'gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/fastqs/mp933-MV411_MEF2D_NT_SC_63_DMSO-MED1-r3_R2_001.fastq.gz',
 'gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/fastqs/mp934-MV411_MEF2D_NT_SC_63_VHL_2h-MED1-r3_R1_001.fastq.gz',
 'gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/fastqs/mp934-MV411_MEF2D_NT_SC_63_VHL_2h-MED1-r3_R2_001.fastq.gz',
 'gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/fastqs/

In [30]:
df = {
"fastq_1": [],
"fastq_2": [],
"antibody": [],
"group": [],
"replicate": [],
"control": []
}
for val in h.grouped(fastq,2):
    row = gsheet[gsheet.id==val[0].split('/')[-1].split('-')[0]]
    df['group'].append("MV4"+row['name'].values[0].split('-r')[0].split('-MV4')[1])
    df['replicate'].append(df['group'].count(df['group'][-1]))
    df['fastq_1'].append(val[0])
    df['fastq_2'].append(val[1])
    df['antibody'].append(row['protein'].values[0])
    df['control'].append("INPUT")
df['group'].append('INPUT')
df['replicate'].append(1)
df['fastq_1'].append('gs://amlproject/Chip/IRF2BP2_degraded_rep3/fastqs/mp845-MV411-INPUT-r2_R1.fastq.gz')
df['fastq_2'].append('gs://amlproject/Chip/IRF2BP2_degraded_rep3/fastqs/mp845-MV411-INPUT-r2_R2.fastq.gz')
df['antibody'].append("")
df['control'].append("")
df = pd.DataFrame(df)
df

,fastq_1,fastq_2,antibody,group,replicate,control
0,gs://amlproject/Chip/chipseq_MEF2C_2h_degraded...,gs://amlproject/Chip/chipseq_MEF2C_2h_degraded...,FLAG_MEF2D,MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D,1,INPUT
1,gs://amlproject/Chip/chipseq_MEF2C_2h_degraded...,gs://amlproject/Chip/chipseq_MEF2C_2h_degraded...,FLAG_MEF2D,MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D,1,INPUT
2,gs://amlproject/Chip/chipseq_MEF2C_2h_degraded...,gs://amlproject/Chip/chipseq_MEF2C_2h_degraded...,MED1,MV411_MEF2D_NT_SC_63_DMSO-MED1,1,INPUT
3,gs://amlproject/Chip/chipseq_MEF2C_2h_degraded...,gs://amlproject/Chip/chipseq_MEF2C_2h_degraded...,MED1,MV411_MEF2D_NT_SC_63_VHL_2h-MED1,1,INPUT
4,gs://amlproject/Chip/chipseq_MEF2C_2h_degraded...,gs://amlproject/Chip/chipseq_MEF2C_2h_degraded...,MEF2C,MV411_MEF2D_NT_SC_63_DMSO-MEF2C,1,INPUT
5,gs://amlproject/Chip/chipseq_MEF2C_2h_degraded...,gs://amlproject/Chip/chipseq_MEF2C_2h_degraded...,MEF2C,MV411_MEF2D_NT_SC_63_VHL_2h-MEF2C,1,INPUT
6,gs://amlproject/Chip/chipseq_MEF2C_2h_degraded...,gs://amlproject/Chip/chipseq_MEF2C_2h_degraded...,MYC,MV411_MEF2D_NT_SC_63_DMSO-MYC,1,INPUT
7,gs://amlproject/Chip/chipseq_MEF2C_2h_degraded...,gs://amlproject/Chip/chipseq_MEF2C_2h_degraded...,MYC,MV411_MEF2D_NT_SC_63_VHL_2h-MYC,1,INPUT
8,gs://amlproject/Chip/chipseq_MEF2C_2h_degraded...,gs://amlproject/Chip/chipseq_MEF2C_2h_degraded...,POLII_total,MV411_MEF2D_NT_SC_63_DMSO-POLII_total,1,INPUT
9,gs://amlproject/Chip/chipseq_MEF2C_2h_degraded...,gs://amlproject/Chip/chipseq_MEF2C_2h_degraded...,POLII_total,MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total,1,INPUT


In [31]:
df[df.columns[[3,4,0,1,2,5]]].to_csv('../nextflow/'+project+'_design.csv',index=False)

In [32]:
project

'chipseq_MEF2C_2h_degraded'

### about

0. you need to have a google project set up with a billing account
1. you need to activte your APIs this way: https://cloud.google.com/life-sciences/docs/tutorials/nextflow?hl=fr
2. nextflow needs to be installed with this installation command `export NXF_MODE=google && curl https://get.nextflow.io | bash`

In [ ]:
! cd ../../ && ./nextflow log ## to get access to the previous runs

In [ ]:
#process chips 
! cd ../../ && ./nextflow run jkobject/chipseq --paired_end --seq_center 'DFCI' --email 'jkobject@gmail.com' --input AMLproject/nextflow/chipseq_MEF2C_2h_degraded_design.csv --genome GRCh38 --max_cpus 16 --skip_diff_analysis -profile jkcloud -w gs://workamlproject/MEF2C2hres --spiking --outdir gs://workamlproject/MEF2C2h --tracedir AMLproject/nextflow/MEF2Cdegraded2h/

In [34]:
! gsutil -m cp -r gs://workamlproject/MEF2C2h/* gs://amlproject/Chip/$project/

Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam.bai [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1.mLb.clN.sorted.bam [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1.mLb.clN.sorted.bam.bai [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2.mLb.clN.sorted.bam [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2.mLb.clN.sorted.bam.bai [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1.mLb.clN.sorted.bam [Conten

Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R2.bigWig [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2.bigWig [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_VHL_2h-MEF2C_R1.bigWig [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_VHL_2h-MEF2C_R2.bigWig [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R1.bigWig [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/bigwig/scale/MV411_MEF2D_NT_SC_63_DMSO-MED1_R2.scale_factor.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/bigwig/scale/MV411_MEF2D_NT_SC_63_DMSO-MYC_R1.scal

Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1.plotFingerprint.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1.plotFingerprint.qcmetrics.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1.plotFingerprint.raw.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MEF2D_NT_SC_63_DMSO-MED1_R2.plotFingerprint.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MEF2D_NT_SC_63_DMSO-MED1_R2.plotFingerprint.qcmetrics.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MEF2D

Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2.plotFingerprint.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2.plotFingerprint.raw.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2.plotFingerprint.qcmetrics.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R1.plotFingerprint.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R1.plotFingerprint.qcmetrics.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/deepTools/plotFi

Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/deepTools/plotProfile/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R2.plotHeatmap.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/deepTools/plotProfile/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R2.plotProfile.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/deepTools/plotProfile/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R2.plotProfile.tab [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/deepTools/plotProfile/MV411_MEF2D_NT_SC_63_DMSO-MYC_R1.computeMatrix.mat.gz [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/deepTools/plotProfile/MV411_MEF2D_NT_SC_63_DMSO-MYC_R1.computeMatrix.vals.mat.tab [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/bwa/mergedLibrary/deepTools/plotProfile/MV411_MEF2D_NT_SC_63_DMSO-MYC_R1.plotHeatmap.pdf [Content-Type

...

Copying gs://workamlproject/MEF2C2h/droso_aligned/counts/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R1_T1_counts.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/droso_aligned/scale/INPUT_R1_T1_scaling.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/droso_aligned/counts/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R2_T1_counts.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/droso_aligned/scale/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2_T1_scaling.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/droso_aligned/scale/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1_T1_scaling.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/droso_aligned/scale/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1_T1_scaling.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/droso_aligned/scale/MV411_MEF2D_NT_SC_63_DMSO-MED1_R2_T1_scaling.txt [C

Copying gs://workamlproject/MEF2C2h/fastqc/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2_T1_1_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/fastqc/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2_T1_2_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/fastqc/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R1_T1_1_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/fastqc/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R1_T1_2_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/fastqc/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R2_T1_1_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/fastqc/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R2_T1_2_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/fastqc/zips/INPUT_R1_T1_1_fastqc.zip [Content-Type=application/octet-stream]...
Copying gs://workamlproj

Copying gs://workamlproject/MEF2C2h/trim_galore/fastqc/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1_T1_1_val_1_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/trim_galore/fastqc/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1_T1_2_val_2_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/trim_galore/fastqc/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2_T1_2_val_2_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/trim_galore/fastqc/MV411_MEF2D_NT_SC_63_DMSO-MED1_R2_T1_1_val_1_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/trim_galore/fastqc/MV411_MEF2D_NT_SC_63_DMSO-MED1_R2_T1_2_val_2_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/trim_galore/fastqc/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R1_T1_1_val_1_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/trim_galore/fastqc/MV411_ME

Copying gs://workamlproject/MEF2C2h/trim_galore/fastqc/zips/MV411_MEF2D_NT_SC_63_DMSO-MYC_R2_T1_1_val_1_fastqc.zip [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/trim_galore/fastqc/zips/MV411_MEF2D_NT_SC_63_DMSO-MYC_R2_T1_2_val_2_fastqc.zip [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/trim_galore/fastqc/zips/MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R1_T1_2_val_2_fastqc.zip [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/trim_galore/fastqc/zips/MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R1_T1_1_val_1_fastqc.zip [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/trim_galore/fastqc/zips/MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R2_T1_1_val_1_fastqc.zip [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/trim_galore/fastqc/zips/MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R2_T1_2_val_2_fastqc.zip [Content-Type=application/octet-stream]...
Copying gs://workaml

Copying gs://workamlproject/MEF2C2h/trim_galore/logs/MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R1_T1_2.fastq.gz_trimming_report.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/trim_galore/logs/MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R2_T1_2.fastq.gz_trimming_report.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/trim_galore/logs/MV411_MEF2D_NT_SC_63_VHL_2h-MEF2C_R1_T1_1.fastq.gz_trimming_report.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/trim_galore/logs/MV411_MEF2D_NT_SC_63_VHL_2h-MEF2C_R1_T1_2.fastq.gz_trimming_report.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/trim_galore/logs/MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R2_T1_1.fastq.gz_trimming_report.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MEF2C2h/trim_galore/logs/MV411_MEF2D_NT_SC_63_VHL_2h-MEF2C_R2_T1_1.fastq.gz_trimming_report.txt [Content-Type=application/octet-stream]...
C

In [35]:
! gsutil cat gs://amlproject/Chip/$project/droso_aligned/counts/*
! gsutil ls gs://amlproject/Chip/$project/droso_aligned/counts/

4587476
1168152
721324
420300
5114472
1606890
2648124
3756150
4138308
1777570
1590322
3797264
501640
2434392
3954514
4706090
658134
1714968
5993438
1886382
6468560
gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/droso_aligned/counts/INPUT_R1_T1_counts.txt
gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/droso_aligned/counts/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1_T1_counts.txt
gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/droso_aligned/counts/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2_T1_counts.txt
gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/droso_aligned/counts/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1_T1_counts.txt
gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/droso_aligned/counts/MV411_MEF2D_NT_SC_63_DMSO-MED1_R2_T1_counts.txt
gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/droso_aligned/counts/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R1_T1_counts.txt
gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/droso_aligned/counts/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R2_T1_counts.txt
gs://amlproject/Chip/chipseq_MEF2C_

In [36]:
project

'chipseq_MEF2C_2h_degraded'

In [ ]:
# now use the total counts in gs://amlproject/Chip/chipseq_MF2C_degraded_v2/multiqc/broadPeak/multiqc_report.html
# https://console.cloud.google.com/storage/browser/amlproject/Chip/chipseq_MF2C_degraded_v2/multiqc/broadPeak/multiqc_report.html

In [37]:
h.createFoldersFor('../../data/'+project+'/bwa/mergedLibrary/')
!gsutil -m cp gs://amlproject/Chip/$project/bwa/mergedLibrary/*.ba* ../../data/$project/bwa/mergedLibrary/
! gsutil -m cp -r gs://amlproject/Chip/$project/bwa/mergedLibrary/bigwig/ ../../data/$project/bwa/mergedLibrary/
! gsutil  -m cp -r gs://amlproject/Chip/$project/bwa/mergedLibrary/macs/ ../../data/$project/bwa/mergedLibrary/

Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam.bai...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1.mLb.clN.sorted.bam.bai...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2.mLb.clN.sorted.bam.bai...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1.mLb.

Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/scale/INPUT_R1.scale_factor.txt...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/scale/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1.scale_factor.txt...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/scale/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2.scale_factor.txt...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/scale/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1.scale_factor.txt...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/scale/MV411_MEF2D_NT_SC_63_DMSO-MED1_R2.scale_factor.txt...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/scale/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R1.scale_factor.txt...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/scale/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R2.scale_factor.txt...
Copying gs://amlproject/Chip/ch

Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R1_peaks.xls...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R2_peaks.annotatePeaks.txt...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R2_peaks.broadPeak...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R1_peaks.xls...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R2_peaks.gappedPeak...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R2_peaks.annotatePeaks.txt...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/MV411_MEF2D_NT_SC_63_DMSO-POL

Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/consensus/MYC/MYC.consensus_peaks.boolean.intersect.txt...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/consensus/MYC/MYC.consensus_peaks.featureCounts.txt...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/consensus/MYC/MYC.consensus_peaks.boolean.txt...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/consensus/MYC/MYC.consensus_peaks.saf...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/consensus/MYC/MYC.consensus_peaks.featureCounts.txt.summary...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/consensus/POLII_total/POLII_total.consensus_peaks.annotatePeaks.txt...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/consensus/POLII_total/POLII_total.consensus_pe

Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/qc/macs_peak.plots.pdf...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/qc/macs_peak.summary.txt...
| [170/170 files][248.6 MiB/248.6 MiB] 100% Done                                
Operation completed over 170 objects/248.6 MiB.                                  


computing scales from the excel sheet

In [42]:
scales = [
1.00,
1.00,
1.00,
0.70,
1.00,
0.10,
0.38,
1.00,
0.92,
1.00,
    
0.26,
0.83,
0.12,
1.00,
0.39,
1.00,
1.00,
0.40,
1.00,
0.03,
         ]

In [43]:
bams = ! ls ../../data/$project/bwa/mergedLibrary/*.bam
bams

['../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1.mLb.clN.sorted.bam',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2.mLb.clN.sorted.bam',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1.mLb.clN.sorted.bam',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-MED1_R2.mLb.clN.sorted.bam',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R1.mLb.clN.sorted.bam',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R2.mLb.clN.sorted.bam',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-MYC_R1.mLb.clN.sorted.bam',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-MYC_R2.mLb.clN.sorted.bam',
 '../

### on scalled data

In [44]:
! mkdir ../../data/$project/diffPeaks/ && ! mkdir ../../data/$project/diffData/

mkdir: cannot create directory ‘../../data/chipseq_MEF2C_2h_degraded/diffPeaks/’: File exists


In [41]:
wigs = ! ls ../../data/$project/bwa/mergedLibrary/bigwig/*.bigWig
wigs

['../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/INPUT_R1.bigWig',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1.bigWig',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2.bigWig',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1.bigWig',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_DMSO-MED1_R2.bigWig',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R1.bigWig',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R2.bigWig',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_DMSO-MYC_R1.bigWig',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_DMSO-MYC_R2.bigWig',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedL

In [45]:
# diffPeak on scaled data (full reprocessing)
for i in range(int(len(bams)/2)):
    if i < 0:
        continue
    bam1 = bams[1+i]
    bam2 = bams[11+i]
    print(bam1,bam2)
    print(chip.fullDiffPeak(bam1,bam2, control1=bams[0], scaling=[scales[i], scales[i+10]], directory = "../../data/"+project+"/diffData/", res_directory = "../../data/"+project+"/diffPeaks/",pairedend=True))

../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1.mLb.clN.sorted.bam ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1.mLb.clN.sorted.bam
doing diff from ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1.mLb.clN.sorted.bam and ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1.mLb.clN.sorted.bam
computing the fragment avg size
181
computing the scaling values
b'INFO  @ Wed, 13 Jan 2021 19:18:38: \n# Command line: callpeak -B -t ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 181 -n MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1 --outdir ../../data/chipseq_MEF2C_2h_degraded/diffData/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MEF2D_NT_SC_63_DMS

CompletedProcess(args='macs2 bdgdiff --t1 ../../data/chipseq_MEF2C_2h_degraded/diffData/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1_treat_pileup.bdg --c1 ../../data/chipseq_MEF2C_2h_degraded/diffData/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1_control_lambda.bdg --t2 ../../data/chipseq_MEF2C_2h_degraded/diffData/MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1_treat_pileup.bdg --c2 ../../data/chipseq_MEF2C_2h_degraded/diffData/MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1_control_lambda.bdg --d1 32893872 --d2 80698857 -g 60 -l 181 --o-prefix MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1_treat_pileup --outdir ../../data/chipseq_MEF2C_2h_degraded/diffPeaks/', returncode=0, stdout=b'', stderr=b'INFO  @ Wed, 13 Jan 2021 20:11:40: Read and build treatment 1 bedGraph... \nINFO  @ Wed, 13 Jan 2021 20:12:47: Read and build control 1 bedGraph... \nINFO  @ Wed, 13 Jan 2021 20:16:36: Read and build treatment 2 bedGraph... \nINFO  @ Wed, 13 Jan 2021 20:17:17: Read 

b'INFO  @ Wed, 13 Jan 2021 20:54:15: \n# Command line: callpeak -B -t ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R2.mLb.clN.sorted.bam -c ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 192 -n MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R2 --outdir ../../data/chipseq_MEF2C_2h_degraded/diffData/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R2\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R2.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fra

198
computing the scaling values
b'INFO  @ Wed, 13 Jan 2021 21:34:28: \n# Command line: callpeak -B -t ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 198 -n MV411_MEF2D_NT_SC_63_DMSO-MED1_R1 --outdir ../../data/chipseq_MEF2C_2h_degraded/diffData/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MEF2D_NT_SC_63_DMSO-MED1_R1\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fr

b'INFO  @ Wed, 13 Jan 2021 22:02:17: \n# Command line: callpeak -B -t ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 198 -n MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R1 --outdir ../../data/chipseq_MEF2C_2h_degraded/diffData/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R1\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n# Lar

...

b'INFO  @ Thu, 14 Jan 2021 03:52:51: \n# Command line: callpeak -B -t ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2.mLb.clN.sorted.bam -c ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 198 -n MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2 --outdir ../../data/chipseq_MEF2C_2h_degraded/diffData/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n# Larger 

208
computing the scaling values
b'INFO  @ Thu, 14 Jan 2021 04:33:49: \n# Command line: callpeak -B -t ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 208 -n MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R1 --outdir ../../data/chipseq_MEF2C_2h_degraded/diffData/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R1\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is 

b'INFO  @ Thu, 14 Jan 2021 05:01:43: \n# Command line: callpeak -B -t ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 208 -n MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R1 --outdir ../../data/chipseq_MEF2C_2h_degraded/diffData/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R1\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted

CompletedProcess(args='macs2 bdgdiff --t1 ../../data/chipseq_MEF2C_2h_degraded/diffData/MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R1_treat_pileup.bdg --c1 ../../data/chipseq_MEF2C_2h_degraded/diffData/MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R1_control_lambda.bdg --t2 ../../data/chipseq_MEF2C_2h_degraded/diffData/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R1_treat_pileup.bdg --c2 ../../data/chipseq_MEF2C_2h_degraded/diffData/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R1_control_lambda.bdg --d1 43923166 --d2 46290409 -g 60 -l 208 --o-prefix MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R1_treat_pileup --outdir ../../data/chipseq_MEF2C_2h_degraded/diffPeaks/', returncode=0, stdout=b'', stderr=b'INFO  @ Thu, 14 Jan 2021 05:31:14: Read and build treatment 1 bedGraph... \nINFO  @ Thu, 14 Jan 2021 05:32:29: Read and build control 1 bedGraph... \nINFO  @ Thu, 14 Jan 2021 05:36:06: Read and build treatment 2 bedGraph... \nINFO  @ Thu, 14 Jan 2021 05:37:29:

b'INFO  @ Thu, 14 Jan 2021 06:15:57: \n# Command line: callpeak -B -t ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R2.mLb.clN.sorted.bam -c ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 208 -n MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R2 --outdir ../../data/chipseq_MEF2C_2h_degraded/diffData/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R2\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R2.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted

In [46]:
initscales = ! cat ../../data/$project/bwa/mergedLibrary/bigwig/scale/*.txt
initscales

['0.00744797',
 '0.0150981',
 '0.0523276',
 '0.0127586',
 '0.033912',
 '0.0181856',
 '0.108942',
 '0.0179894',
 '0.0308957',
 '0.0123208',
 '0.033895',
 '0.0236527',
 '0.094506',
 '0.0185945',
 '0.0294184',
 '0.0201126',
 '0.0364321',
 '0.0165658',
 '0.0626348',
 '0.010742',
 '0.0341605']

In [47]:
rescales = [val*float(initscales[1+i]) for i, val in enumerate(scales)]
rescales

[0.0150981,
 0.0523276,
 0.0127586,
 0.023738399999999996,
 0.0181856,
 0.0108942,
 0.006835972,
 0.0308957,
 0.011335136,
 0.033895,
 0.006149702,
 0.07843998,
 0.00223134,
 0.0294184,
 0.007843914,
 0.0364321,
 0.0165658,
 0.025053920000000004,
 0.010742,
 0.001024815]

In [48]:
chip.bigWigFrom(bams[1:], 
                genome='GRCh38',scaling=rescales,
               numthreads=8)

In [49]:
!mkdir ../../data/$project/recalib_bigwig/

In [50]:
!mv bigwig/* ../../data/$project/recalib_bigwig/

In [73]:
! gsutil -m cp -r ../../data/$project/recalib_bigwig gs://amlproject/Chip/$project/

Copying file://../../data/chipseq_MEF2C_2h_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R1.bw [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R2.bw [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file://../../

In [51]:
os.popen('for i in $(ls ../../data/'+project+'/diffPeaks/*.bed); \
            do echo $(wc -l $i); \
            done').read().split('\n')

['218 ../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1_treat_pileup_c3.0_common.bed',
 '4257 ../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1_treat_pileup_c3.0_cond1.bed',
 '6 ../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1_treat_pileup_c3.0_cond2.bed',
 '146 ../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R2_treat_pileup_c3.0_common.bed',
 '14 ../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R2_treat_pileup_c3.0_cond1.bed',
 '212 ../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2_tr

In [52]:
bw = ! ls ../../data/$project/recalib_bigwig/*
bw

['../../data/chipseq_MEF2C_2h_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1.bw',
 '../../data/chipseq_MEF2C_2h_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2.bw',
 '../../data/chipseq_MEF2C_2h_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1.bw',
 '../../data/chipseq_MEF2C_2h_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_DMSO-MED1_R2.bw',
 '../../data/chipseq_MEF2C_2h_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R1.bw',
 '../../data/chipseq_MEF2C_2h_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R2.bw',
 '../../data/chipseq_MEF2C_2h_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_DMSO-MYC_R1.bw',
 '../../data/chipseq_MEF2C_2h_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_DMSO-MYC_R2.bw',
 '../../data/chipseq_MEF2C_2h_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R1.bw',
 '../../data/chipseq_MEF2C_2h_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R2.bw',
 '../../data/chipseq_MEF2C_2h_degraded/recal

In [53]:
!mkdir ../results/$project/
!mkdir ../results/$project/plots
!mkdir ../results/$project/plots/heatmaps/

In [54]:
peaks = ! ls ../../data/$project/bwa/mergedLibrary/macs/broadPeak/*.broadPeak
peaks

['../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1_peaks.broadPeak',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2_peaks.broadPeak',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1_peaks.broadPeak',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/MV411_MEF2D_NT_SC_63_DMSO-MED1_R2_peaks.broadPeak',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R1_peaks.broadPeak',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R2_peaks.broadPeak',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/MV411_MEF2D_NT_SC_63_DMSO-MYC_R1_peaks.broadPeak',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/macs/broadPeak/MV411_MEF2D_NT_SC_63_DMSO-MYC_R2_peaks.broadPea

In [56]:
names = ["FLAG_MEF2D_R1", "FLAG_MEF2D_R2", "MED1_R1", "MED1_R2", "MEF2C_R1", "MEF2C_R2", "MYC_R1", "MYC_R2", "POLII_total_R1", "POLII_total_R2",]

#### merging peaks VHL/DMSO

In [57]:
mpeaks = []
for i, val in enumerate(names):
    if i<0:
        continue
    print(val)
    dmso = peaks[i]
    vhl = peaks[i+10]
    chip.simpleMergePeaks(pd.concat([chip.loadPeaks(dmso), chip.loadPeaks(vhl)])).to_csv('../../data/' + project + '/' + val + '_genomewide_merged.bed', sep='\t', header=False, index=False)
    mpeaks.append('../../data/'+project+'/'+val+'_genomewide_merged.bed')

FLAG_MEF2D_R1
FLAG_MEF2D_R2629846643
MED1_R17997557997664
MED1_R2399183951045444
MEF2C_R19319041615445
MEF2C_R26452029719665
MYC_R19676511954995657
MYC_R27788393489034465
POLII_total_R153186453
POLII_total_R279717455


#### GENOME WIDE comparison

In [61]:
dmsoname="_DMSO"
condname="_wMEF2D_ko_at2h"

In [65]:
for i, val in enumerate(bw):
    if i > 0:
        continue
    name = names[i-10]+condname if i//10 else names[i]+dmsoname
    print(name)
    chip.getPeaksAt(mpeaks[i%10], bigwigs = val, bigwignames= name, peaknames=['Macs2_Peaks'], window=3000, folder="", title=name, numthreads=8, refpoint="center", name='../../data/'+project+'/'+name+'_mat.pdf', withDeeptools=True, torecompute=True, legendLoc="lower-left")

FLAG_MEF2D_R1_DMSO
CompletedProcess(args='computeMatrix reference-point -S ../../data/chipseq_MEF2C_2h_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1.bw  --referencePoint center --regionsFileName ../../data/chipseq_MEF2C_2h_degraded/FLAG_MEF2D_R1_genomewide_merged.bed  --missingDataAsZero --outFileName ../../data/chipseq_MEF2C_2h_degraded/FLAG_MEF2D_R1_DMSO_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 8 && plotHeatmap --matrixFile ../../data/chipseq_MEF2C_2h_degraded/FLAG_MEF2D_R1_DMSO_mat.gz --outFileName ../../data/chipseq_MEF2C_2h_degraded/FLAG_MEF2D_R1_DMSO_mat.pdf --refPointLabel center --legendLocation lower-left --regionsLabel Macs2_Peaks --plotTitle FLAG_MEF2D_R1_DMSO', returncode=0, stdout=b'', stderr=b'')


#### making overlapping profiles

In [66]:
for i, val in enumerate(names):
    if i <0:
        continue
    val1 = '../../data/'+project+'/' + val + dmsoname+'_mat.gz'
    val2 = '../../data/'+project+'/' + val + condname+'_mat.gz'
    print(val)
    chip.makeProfiles(matx=[val1,val2], matnames=['DMSO','VHL'], title=val, refpoint="center", name='../../data/'+project+'/'+val+'_combined_mat.pdf', legendLoc="lower-left")

FLAG_MEF2D_R1
CompletedProcess(args='computeMatrixOperations relabel -m ../../data/chipseq_MEF2C_2h_degraded/FLAG_MEF2D_R1_DMSO_mat.gz -o ../../data/chipseq_MEF2C_2h_degraded/FLAG_MEF2D_R1_DMSO_mat.gz --groupLabels DMSO && computeMatrixOperations relabel -m ../../data/chipseq_MEF2C_2h_degraded/FLAG_MEF2D_R1_wMEF2D_ko_at2h_mat.gz -o ../../data/chipseq_MEF2C_2h_degraded/FLAG_MEF2D_R1_wMEF2D_ko_at2h_mat.gz --groupLabels VHL && computeMatrixOperations rbind -m ../../data/chipseq_MEF2C_2h_degraded/FLAG_MEF2D_R1_DMSO_mat.gz ../../data/chipseq_MEF2C_2h_degraded/FLAG_MEF2D_R1_wMEF2D_ko_at2h_mat.gz -o ../../data/chipseq_MEF2C_2h_degraded/FLAG_MEF2D_R1_combined_mat.gz && plotProfile --matrixFile ../../data/chipseq_MEF2C_2h_degraded/FLAG_MEF2D_R1_combined_mat.gz --outFileName ../../data/chipseq_MEF2C_2h_degraded/FLAG_MEF2D_R1_combined_mat.pdf --refPointLabel center --legendLocation lower-left --plotTitle FLAG_MEF2D_R1', returncode=0, stdout=b'', stderr=b'')
FLAG_MEF2D_R2
CompletedProcess(args='co

CompletedProcess(args='computeMatrixOperations relabel -m ../../data/chipseq_MEF2C_2h_degraded/POLII_total_R2_DMSO_mat.gz -o ../../data/chipseq_MEF2C_2h_degraded/POLII_total_R2_DMSO_mat.gz --groupLabels DMSO && computeMatrixOperations relabel -m ../../data/chipseq_MEF2C_2h_degraded/POLII_total_R2_wMEF2D_ko_at2h_mat.gz -o ../../data/chipseq_MEF2C_2h_degraded/POLII_total_R2_wMEF2D_ko_at2h_mat.gz --groupLabels VHL && computeMatrixOperations rbind -m ../../data/chipseq_MEF2C_2h_degraded/POLII_total_R2_DMSO_mat.gz ../../data/chipseq_MEF2C_2h_degraded/POLII_total_R2_wMEF2D_ko_at2h_mat.gz -o ../../data/chipseq_MEF2C_2h_degraded/POLII_total_R2_combined_mat.gz && plotProfile --matrixFile ../../data/chipseq_MEF2C_2h_degraded/POLII_total_R2_combined_mat.gz --outFileName ../../data/chipseq_MEF2C_2h_degraded/POLII_total_R2_combined_mat.pdf --refPointLabel center --legendLocation lower-left --plotTitle POLII_total_R2', returncode=0, stdout=b'', stderr=b'')


In [67]:
h.createFoldersFor('../results/'+project+'/plots/scaled/heatmaps/')
! cp ../../data/$project/*.pdf ../results/$project/plots/scaled/heatmaps/

In [68]:
cond1peak = ! ls ../../data/$project/diffPeaks/*cond1.bed
cond2peak = ! ls ../../data/$project/diffPeaks/*cond2.bed
commonpeak = ! ls ../../data/$project/diffPeaks/*common.bed
cond1peak

['../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1_treat_pileup_c3.0_cond1.bed',
 '../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R2_treat_pileup_c3.0_cond1.bed',
 '../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R1_treat_pileup_c3.0_cond1.bed',
 '../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-MED1_R2_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R2_treat_pileup_c3.0_cond1.bed',
 '../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-MEF2C_R1_treat_pileup_c3.0_cond1.bed',
 '../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R2_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-MEF2C_R2_treat_pileup

In [69]:
for i in range(int(len(bw)/2)):
    if i <0:
        continue
    name1 = bw[i]
    name2 = bw[i+len(names)]
    peak = [cond1peak[i], commonpeak[i], cond2peak[i]]
   # for val in peak:
      #  chip.dropWeirdChromosomes(val)
    name = names[i]
    print(name1,name2)
    chip.getPeaksAt(peak, [name1, name2], bigwignames=['DMSO', 'VHL'], peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], window=3000, folder="", title=name, numthreads=8, refpoint="center", name='../../data/'+project+'/diffPeaks/'+name+'_mat.pdf', withDeeptools=True, torecompute=False)

../../data/chipseq_MEF2C_2h_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1.bw ../../data/chipseq_MEF2C_2h_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1.bw
CompletedProcess(args='computeMatrix reference-point -S ../../data/chipseq_MEF2C_2h_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1.bw ../../data/chipseq_MEF2C_2h_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1.bw  --referencePoint center --regionsFileName ../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1_treat_pileup_c3.0_cond1.bed ../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1_treat_pileup_c3.0_common.bed  --missingDataAsZero --outFileName ../../data/chipseq_MEF2C_2h_degraded/diffPeaks/FLAG_MEF2D_R1_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 8 && plotHea

CompletedProcess(args='computeMatrix reference-point -S ../../data/chipseq_MEF2C_2h_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_DMSO-MYC_R2.bw ../../data/chipseq_MEF2C_2h_degraded/recalib_bigwig/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2.bw  --referencePoint center --regionsFileName ../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-MYC_R2_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2_treat_pileup_c3.0_cond1.bed ../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-MYC_R2_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2_treat_pileup_c3.0_common.bed  --missingDataAsZero --outFileName ../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MYC_R2_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 8 && plotHeatmap --matrixFile ../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MYC_R2_mat.gz --outFileName ../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MYC_R2_mat.pdf --refPointLabel center --regionsLabel DMSO_peaks common --plotTitle MYC_R2', returncod

In [70]:
! mkdir ../results/$project/plots/scaled/diffPeaks/
! cp ../../data/$project/diffPeaks/*.pdf ../results/$project/plots/scaled/diffPeaks/

#### Looking at TSS

In [ ]:
IRF8targets = h.fileToList('../results/slamseqMax/IRF8targets.txt')

In [ ]:
ls ../../data/chipseq_IRF8_degraded/recalib_bigwig/*

In [ ]:
peaksVHL = pd.read_csv('../../data/chipseq_IRF8_degraded/diffPeaks/MV411_IRF8_NT_DMSO-POLII_total_R1_treat_pileup_vs_MV411_IRF8_NT_VHL-POLII_total_R1_treat_pileup_c3.0_cond2.bed', sep='\t',header=None, skiprows=1, names=['chrom','start','end','name','score'])
peaksVHL['chrom']= [i[3:] for i in peaksVHL['chrom']]
peaksVHL[['start','end']] = peaksVHL[['start','end']].astype(int)

In [ ]:
peaksDMSO = pd.read_csv('../../data/chipseq_IRF8_degraded/diffPeaks/MV411_IRF8_NT_DMSO-POLII_total_R1_treat_pileup_vs_MV411_IRF8_NT_VHL-POLII_total_R1_treat_pileup_c3.0_cond1.bed', sep='\t', header=None, skiprows=1, names=['chrom','start','end','name','score'])
peaksDMSO['chrom']= [i[3:] for i in peaksDMSO['chrom']]
peaksDMSO[['start','end']] = peaksDMSO[['start','end']].astype(int)

In [ ]:
peakcommon = pd.read_csv('../../data/chipseq_IRF8_degraded/diffPeaks/MV411_IRF8_NT_DMSO-POLII_total_R1_treat_pileup_vs_MV411_IRF8_NT_VHL-POLII_total_R1_treat_pileup_c3.0_common.bed', sep='\t',header=None, skiprows=1, names=['chrom','start','end','name','score'])
peakcommon['chrom']= [i[3:] for i in peakcommon['chrom']]
peakcommon[['start','end']] = peakcommon[['start','end']].astype(int)

In [ ]:
ensembltss['Chromosome/scaffold name'] = ensembltss['Chromosome/scaffold name'].astype(str)
ensembltss = ensembltss[ensembltss['Chromosome/scaffold name'].isin(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '3', '4', '5', '6', '7', '8', '9','X','Y'])]
ensembltss = ensembltss.drop_duplicates('Gene start (bp)')
ensembltss = ensembltss.reset_index(drop=True)
ensembltss = ensembltss.astype(str)
ensembltss['loci'] = ensembltss['Transcription start site (TSS)'].astype(int)
ensembltss['chrom'] = ensembltss['Chromosome/scaffold name']
ensembltss = ensembltss.sort_values(by=['chrom','loci']).reset_index(drop=True)

In [ ]:
chip.substractPeaksTo(peaksDMSO, ensembltss).to_csv('../../data/'+project+'/diffPeaks/polII_DMSO_TSSonly.bed',sep='\t',index=None,header=False)
chip.substractPeaksTo(peakcommon, ensembltss).to_csv('../../data/'+project+'/diffPeaks/polII_common_TSSonly.bed',sep='\t',index=None,header=False)
chip.substractPeaksTo(peaksVHL, ensembltss).to_csv('../../data/'+project+'/diffPeaks/polII_VHL_TSSonly.bed',sep='\t',index=None,header=False)

In [ ]:
chip.getPeaksAt(['../../data/'+project+'/diffPeaks/polII_DMSO_TSSonly.bed', '../../data/'+project+'/diffPeaks/polII_common_TSSonly.bed', '../../data/'+project+'/diffPeaks/polII_VHL_TSSonly.bed'],
                bigwigs = ['../../data/chipseq_IRF8_degraded/recalib_bigwig/MV411_IRF8_NT_DMSO-POLII_total_R1.bw', '../../data/chipseq_IRF8_degraded/recalib_bigwig/MV411_IRF8_NT_VHL-POLII_total_R1.bw'], 
                bigwignames= ['IRF8_DMSO', 'IRF8_VHL'], 
                peaknames=['DMSO','common','VHL'], window=3000, folder="", title='polII_with_IRF8_degron', numthreads=4, 
                refpoint="center", 
                name='../../data/'+project+'/diffPeaks/polII_with_IRF8_degron_mat.pdf', 
                withDeeptools=True, torecompute=True, legendLoc="lower-left")

In [ ]:
IRF8tss = ensembltss[ensembltss['Gene name'].isin(IRF8targets)].reset_index(drop=True)

In [ ]:
a = ! wc -l ../../data/$project/diffPeaks/polII_DMSO_TSSonly.bed 

In [ ]:
int(a[0].split(' ')[0])/len(peaksDMSO), len(peaksDMSO)

In [ ]:
len(chip.substractPeaksTo(peaksDMSO, IRF8tss))/int(a[0].split(' ')[0])

In [ ]:
a = ! wc -l ../../data/$project/diffPeaks/polII_common_TSSonly.bed 

In [ ]:
int(a[0].split(' ')[0])/ len(peakcommon),  len(peakcommon)

In [ ]:
len(chip.substractPeaksTo(peakcommon, IRF8tss))/int(a[0].split(' ')[0])

In [ ]:
a = ! wc -l ../../data/$project/diffPeaks/polII_VHL_TSSonly.bed 

In [ ]:
len(peaksVHL)

In [ ]:
int(a[0].split(' ')[0])/len(peaksVHL),  len(peaksVHL)

In [ ]:
len(chip.substractPeaksTo(peaksVHL, IRF8tss))/int(a[0].split(' ')[0])

### on unscalled data

In [71]:
! mkdir ../../data/$project/diffPeaks_unscaled

In [74]:
#on unscalled data 
for i in range(int(len(bams)/2)):
    if i < 0:
        continue
    name1 = bams[1+i]
    name2 = bams[1+len(names)+i]
    print(name1,name2)
    chip.fullDiffPeak(name1,name2, control1='../../data/'+project+'/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam', directory = "../../data/"+project+"/diffData_unscaled/", res_directory = "../../data/"+project+"/diffPeaks_unscaled/",pairedend=False)

../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1.mLb.clN.sorted.bam ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1.mLb.clN.sorted.bam
doing diff from ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1.mLb.clN.sorted.bam and ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1.mLb.clN.sorted.bam
computing the fragment avg size
181
computing the scaling values
b'INFO  @ Thu, 14 Jan 2021 20:59:00: \n# Command line: callpeak -B -t ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 181 -n MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1 --outdir ../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = MV411_MEF2D_NT_SC

b'INFO  @ Thu, 14 Jan 2021 21:14:48: \n# Command line: callpeak -B -t ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 181 -n MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1 --outdir ../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1\n# format = BAM\n# ChIP-seq file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicte

192
computing the scaling values
b'INFO  @ Thu, 14 Jan 2021 21:40:31: \n# Command line: callpeak -B -t ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2.mLb.clN.sorted.bam -c ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 192 -n MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2 --outdir ../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2\n# format = BAM\n# ChIP-seq file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is

../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1.mLb.clN.sorted.bam ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R1.mLb.clN.sorted.bam
doing diff from ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1.mLb.clN.sorted.bam and ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R1.mLb.clN.sorted.bam
computing the fragment avg size
198
computing the scaling values
b'INFO  @ Thu, 14 Jan 2021 22:16:26: \n# Command line: callpeak -B -t ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 198 -n MV411_MEF2D_NT_SC_63_DMSO-MED1_R1 --outdir ../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = MV411_MEF2D_NT_SC_63_DMSO-MED1_R1\n# format = BAM\n# 

b'INFO  @ Thu, 14 Jan 2021 22:32:49: \n# Command line: callpeak -B -t ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 198 -n MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R1 --outdir ../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R1\n# format = BAM\n# ChIP-seq file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n

...

b'INFO  @ Fri, 15 Jan 2021 01:09:35: \n# Command line: callpeak -B -t ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 193 -n MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R1 --outdir ../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R1\n# format = BAM\n# ChIP-seq file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n# La

../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-MYC_R2.mLb.clN.sorted.bam ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2.mLb.clN.sorted.bam
doing diff from ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-MYC_R2.mLb.clN.sorted.bam and ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2.mLb.clN.sorted.bam
computing the fragment avg size
198
computing the scaling values
b'INFO  @ Fri, 15 Jan 2021 01:36:51: \n# Command line: callpeak -B -t ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-MYC_R2.mLb.clN.sorted.bam -c ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 198 -n MV411_MEF2D_NT_SC_63_DMSO-MYC_R2 --outdir ../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = MV411_MEF2D_NT_SC_63_DMSO-MYC_R2\n# format = BAM\n# ChIP-se

b'INFO  @ Fri, 15 Jan 2021 01:50:01: \n# Command line: callpeak -B -t ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2.mLb.clN.sorted.bam -c ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 198 -n MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2 --outdir ../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2\n# format = BAM\n# ChIP-seq file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n# La

b'INFO  @ Fri, 15 Jan 2021 02:30:40: \n# Command line: callpeak -B -t ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 208 -n MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R1 --outdir ../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R1\n# format = BAM\n# ChIP-seq file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the pred

../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R2.mLb.clN.sorted.bam ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R2.mLb.clN.sorted.bam
doing diff from ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R2.mLb.clN.sorted.bam and ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R2.mLb.clN.sorted.bam
computing the fragment avg size
208
computing the scaling values
b'INFO  @ Fri, 15 Jan 2021 03:14:50: \n# Command line: callpeak -B -t ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R2.mLb.clN.sorted.bam -c ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 208 -n MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R2 --outdir ../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/ -f BAM\n# ARGUMENTS LIST:\n# name = MV411_M

In [75]:
bw = ! ls ../../data/$project/bwa/mergedLibrary/bigwig/*.bigWig
bw

['../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/INPUT_R1.bigWig',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1.bigWig',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2.bigWig',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1.bigWig',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_DMSO-MED1_R2.bigWig',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R1.bigWig',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R2.bigWig',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_DMSO-MYC_R1.bigWig',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_DMSO-MYC_R2.bigWig',
 '../../data/chipseq_MEF2C_2h_degraded/bwa/mergedL

In [76]:
cond1peak = ! ls ../../data/$project/diffPeaks_unscaled/*cond1.bed
cond2peak = ! ls ../../data/$project/diffPeaks_unscaled/*cond2.bed
commonpeak = ! ls ../../data/$project/diffPeaks_unscaled/*common.bed
commonpeak

['../../data/chipseq_MEF2C_2h_degraded/diffPeaks_unscaled/INPUT_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R2_treat_pileup_c3.0_common.bed',
 '../../data/chipseq_MEF2C_2h_degraded/diffPeaks_unscaled/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1_treat_pileup_c3.0_common.bed',
 '../../data/chipseq_MEF2C_2h_degraded/diffPeaks_unscaled/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R2_treat_pileup_c3.0_common.bed',
 '../../data/chipseq_MEF2C_2h_degraded/diffPeaks_unscaled/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R1_treat_pileup_c3.0_common.bed',
 '../../data/chipseq_MEF2C_2h_degraded/diffPeaks_unscaled/MV411_MEF2D_NT_SC_63_DMSO-MED1_R2_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R2_treat_pileup_c3.0_common.bed',
 '../../data/chipseq_MEF2C_2h_degraded/diffPeaks_unscaled/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R1_treat_pileup_vs_MV411_MEF2D_

In [77]:
for i in range(int((len(bw)-1)/2)):
    name1 = bw[1+i]
    name2 = bw[1+len(names)+i]
    peak = [cond1peak[i], commonpeak[i], cond2peak[i]]
    #for val in peak:
     #   chip.dropWeirdChromosomes(val)
    name = names[i]
    print(name1,name2)
    chip.getPeaksAt(peak, [name1, name2], bigwignames=['DMSO', 'VHL'], peaknames=['DMSO_peaks', 'common', 'VHL_peaks'], window=3000, folder="", title=name, numthreads=8, torecompute=True, refpoint='center', name='../../data/'+project+'/diffPeaks_unscaled/'+name+'_mat.pdf', withDeeptools=True)

../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1.bigWig ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1.bigWig
CompletedProcess(args='computeMatrix reference-point -S ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1.bigWig ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1.bigWig  --referencePoint center --regionsFileName ../../data/chipseq_MEF2C_2h_degraded/diffPeaks_unscaled/INPUT_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R2_treat_pileup_c3.0_cond2.bed  --missingDataAsZero --outFileName ../../data/chipseq_MEF2C_2h_degraded/diffPeaks_unscaled/FLAG_MEF2D_R1_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 8 && plotHeatmap --matrixFile ../../data/chipseq_MEF2C_2h_degraded/diffPeaks_unscaled/FLAG_MEF2D_R1_mat.gz --outFileName ../../data/chipseq_MEF2

CompletedProcess(args='computeMatrix reference-point -S ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R2.bigWig ../../data/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/bigwig/MV411_MEF2D_NT_SC_63_VHL_2h-MEF2C_R2.bigWig  --referencePoint center --regionsFileName ../../data/chipseq_MEF2C_2h_degraded/diffPeaks_unscaled/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-MEF2C_R1_treat_pileup_c3.0_common.bed ../../data/chipseq_MEF2C_2h_degraded/diffPeaks_unscaled/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R1_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-MEF2C_R1_treat_pileup_c3.0_cond2.bed  --missingDataAsZero --outFileName ../../data/chipseq_MEF2C_2h_degraded/diffPeaks_unscaled/MEF2C_R2_mat.gz --upstream 3000 --downstream 3000 --numberOfProcessors 8 && plotHeatmap --matrixFile ../../data/chipseq_MEF2C_2h_degraded/diffPeaks_unscaled/MEF2C_R2_mat.gz --outFileName ../../data/chipseq_MEF2C_2h_degraded/diffPeaks_unscaled/MEF2C_R2_mat

In [78]:
h.createFoldersFor('../results/'+project+'/plots/unscaled/diffPeaks/')
! cp ../../data/$project/diffPeaks_unscaled/*.pdf ../results/$project/plots/unscaled/diffPeaks/

! gsutil -m cp gs://amlproject/Chip/$project/bwa/mergedLibrary/deepTools/**.pdf ../results/$project/plots/

Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1.plotFingerprint.pdf...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2.plotFingerprint.pdf...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1.plotFingerprint.pdf...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MEF2D_NT_SC_63_DMSO-MED1_R2.plotFingerprint.pdf...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R1.plotFingerprint.pdf...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R2.plotFingerprint.pdf...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degr

Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/deepTools/plotProfile/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R1.plotHeatmap.pdf...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/deepTools/plotProfile/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2.plotHeatmap.pdf...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/deepTools/plotProfile/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2.plotProfile.pdf...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/deepTools/plotProfile/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R1.plotHeatmap.pdf...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/deepTools/plotProfile/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R1.plotProfile.pdf...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/deepTools/plotProfile/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R2.plotProfile.pdf...
Copying gs://amlproject/Chip/chipseq_MEF2C_2h_degraded/bwa/mergedLibrary/deepTools

In [79]:
! gsutil -m cp -r ../../data/$project/diffPeaks gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/diffData gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/diffPeaks_unscaled gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/diffData_unscaled gs://amlproject/Chip/$project/
! gsutil -m cp -r ../../data/$project/peakplot gs://amlproject/Chip/$project/

Copying file://../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R2_treat_pileup_c3.0_common.bed [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffPeaks/POLII_total_R1_mat.gz [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-MYC_R2_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2_treat_pileup_c3.0_common.bed [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R2_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R2_treat_pileup_c3.0_cond1.bed [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffPeaks/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R2_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R2_treat_pileup_c3.0_cond2.bed [C

| [50/50 files][108.7 MiB/108.7 MiB] 100% Done                                  
Operation completed over 50 objects/108.7 MiB.                                   
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1_peaks.xls [Content-Type=application/vnd.ms-excel]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData/MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1_treat_pileup.bdg [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R2_summits.bed [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com

Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R1_treat_pileup.bdg [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData/MV411_MEF2D_NT_SC_63_DMSO-MYC_R2_summits.bed [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData/MV411_MEF2D_NT_SC_63_DMSO-MEF2C_R1_summits.bed [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData/MV411_MEF2D_NT_SC_63_DMSO-MYC_R1_treat_pileup.bdg [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData/MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1_control_lambda.bdg [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData/MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R2_peaks.narrowPeak [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData/MV411_M

Copying file://../../data/chipseq_MEF2C_2h_degraded/diffPeaks_unscaled/MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R2_treat_pileup_vs_MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R2_treat_pileup_c3.0_common.bed [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffPeaks_unscaled/MED1_R1_mat.pdf [Content-Type=application/pdf]...
| [53/53 files][112.8 MiB/112.8 MiB] 100% Done                                  
Operation completed over 53 objects/112.8 MiB.                                   
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/MV411_MEF2D_NT_SC_63_DMSO-MED1_R1_peaks.xls [Content-Type=application/vnd.ms-excel]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/INPUT_R1_peaks.xls [Content-Type=application/vnd.ms-excel]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R1_peaks.xls [Content-Type=application/vnd.ms-excel]...
Copying file://../.

Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R1_peaks.narrowPeak [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/MV411_MEF2D_NT_SC_63_VHL_2h-MED1_R2_control_lambda.bdg [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/MV411_MEF2D_NT_SC_63_DMSO-MED1_R2_peaks.xls [Content-Type=application/vnd.ms-excel]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R2_peaks.narrowPeak [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/MV411_MEF2D_NT_SC_63_VHL_2h-MYC_R1_peaks.xls [Content-Type=application/vnd.ms-excel]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/MV411_MEF2D_NT_SC_63_DMSO-MYC_R1_peaks.narrowPeak [Content-Type=application/octet-stream]...
Copying file://../../dat

Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/MV411_MEF2D_NT_SC_63_VHL_2h-MEF2C_R1_summits.bed [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/MV411_MEF2D_NT_SC_63_DMSO-FLAG_MEF2D_R1_summits.bed [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/MV411_MEF2D_NT_SC_63_VHL_2h-FLAG_MEF2D_R2_peaks.xls [Content-Type=application/vnd.ms-excel]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/MV411_MEF2D_NT_SC_63_VHL_2h-POLII_total_R1_peaks.narrowPeak [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/MV411_MEF2D_NT_SC_63_VHL_2h-MEF2C_R1_peaks.narrowPeak [Content-Type=application/octet-stream]...
Copying file://../../data/chipseq_MEF2C_2h_degraded/diffData_unscaled/MV411_MEF2D_NT_SC_63_DMSO-POLII_total_R2_treat_pileup.bdg [Content-Type=application/octet-stream].